# Practice Session 3: From Distributions to Statistical Thinking


In [2]:
import pandas as pd
import numpy as np
from numpy.random import default_rng

import seaborn as sns

from scipy import stats
from scipy.stats import skew, kurtosis, shapiro, kstest, norm

import matplotlib.pyplot as plt

from ipywidgets import interact, FloatSlider
plt.rcParams["figure.figsize"] = (8, 8)

## Part 1: Revisiting the Wine Quality Dataset

Yesterday, we began exploring the **Wine Quality dataset** and learned how to visualize and analyze distributions.  
Before we move further, there are two very important points to revisit.

Let’s take another look at the dataset and recall how summary statistics are reported in Python.  
When we use the **`.describe()`** method, it automatically provides key descriptive measures such as the\
**mean**, **standard deviation**, **minimum**, **maximum**, and **quartiles** for each numeric variable.

However, not all functions in Python compute these statistics in exactly the same way.  
Today, we’ll check this by comparing different implementations of the **standard deviation**.



In [2]:
# Load the Wine Quality dataset
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
wine_data = pd.read_csv(url, sep=";")

In [3]:
wine_data.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000
mean,8.319637,0.527821,0.270976,2.538806,0.087467,15.874922,46.467792,0.996747,3.311113,0.658149,10.422983,5.636023
std,1.741096,0.179060,0.194801,1.409928,0.047065,10.460157,32.895324,0.001887,0.154386,0.169507,1.065668,0.807569
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000,3.000000
25%,7.100000,0.390000,0.090000,1.900000,0.070000,7.000000,22.000000,0.995600,3.210000,0.550000,9.500000,5.000000
50%,7.900000,0.520000,0.260000,2.200000,0.079000,14.000000,38.000000,0.996750,3.310000,0.620000,10.200000,6.000000
75%,9.200000,0.640000,0.420000,2.600000,0.090000,21.000000,62.000000,0.997835,3.400000,0.730000,11.100000,6.000000
max,15.900000,1.580000,1.000000,15.500000,0.611000,72.000000,289.000000,1.003690,4.010000,2.000000,14.900000,8.000000


#### <font color='#fc7202'> Task 1: </font>  
Using the **Wine Quality dataset**, calculate the **standard deviation** for all numeric variables with two different functions:

1. **`numpy.std()`**  
2. **`scipy.stats.tstd()`**

Then, compare the results with the standard deviation values shown in the `.describe()` summary table.  
Are all the results identical? If not, think about why they differ.

In [ ]:
# YOUR CODE HERE!

#### Comparing Variability Across Variables  

As we can see, the standard deviations vary quite a lot between the variables.  
However, since each variable may have a **different mean and scale**, directly comparing their standard deviations isn’t very meaningful.  

To make the comparison fair, we can compute the **coefficient of variation (CV)**, which expresses the **standard deviation relative to the mean**:
\begin{equation} \text {CV} = \frac{s}{\bar{x}} \end{equation}



#### <font color='#fc7202'> Task 2: </font>  
Compute the **coefficient of variation** for each numeric variable in the dataset.  


In [21]:
# YOUR CODE HERE!

#### Revisiting the Kolgomorov-Smirnov (K-S) test

Yesterday we saw that we can use the **Kolmogorov–Smirnov (K–S) test** to check whether our data come from a normal distribution.  
This test compares the **empirical cumulative distribution function (ECDF)** of the data with the **theoretical cumulative distribution function (CDF)** of the normal distribution.

Before we look deeper into the test, let’s first understand what the **CDF** actually represents.

**The Probability Density Function (PDF)**

The **PDF** describes how probability is *distributed* across a continuous variable.  
It shows where the values of a variable are **most likely to occur**, but the **height of the curve** itself is *not* a probability.

> - For continuous data (like height), the probability of any **exact value** (e.g., exactly 165.000... cm) is **zero**.
> - Instead, probability is found by the **area under the curve** between two points.  
>   For example:
>   \begin{equation} P(160 < X < 170) = \int_{160}^{170} f(x)\,dx \end{equation}
>   where $f(x)$ is the PDF.

Because all probabilities together must add up to 1,  
\begin{equation} \int_{-\infty}^{\infty} f(x)\,dx = 1 \end{equation}


**The Cumulative Distribution Function (CDF)**

The **CDF**, usually written as $F(x)$, tells us **the probability that a value is less than or equal to x**:
\begin{equation} F(x) = P(X \le x) = \int_{-\infty}^{x} f(t)\,dt \end{equation}
So, the CDF is literally the **accumulated area under the PDF** up to the point $x$.

- The *y*-axis of the CDF shows this cumulative probability - it always ranges from 0 to 1.
- At very small $x$, almost no area is accumulated, so $F(x) \approx 0$.
- Around the middle (for example, the mean of a normal distribution), $F(x)$ (might be) is around 0.5, meaning half the total probability lies below that point.
- As $x$ grows large, the entire area is included, and $F(x) \to 1$.

> - The PDF is the **derivative** of the CDF:
>   \begin{equation} f(x) = \frac{dF(x)}{dx} \end{equation}
> - The CDF is the **integral** of the PDF:
>   \begin{equation} F(x) = \int_{-\infty}^{x} f(t)\,dt \end{equation}
  

Graphically:
- The PDF shows how *dense* probability is at each point.
- The CDF shows how that density *accumulates* as we move from left to right.

In the cell below, you can interactively change the **threshold** and see, how the area under the PDF up to that threshold corresponds to the value of the CDF at that same point.


In [3]:
np.random.seed(42)

def pdf_cdf_threshold_standard(thresh=0.0):
    # Fixed standard normal distribution
    mu, sigma = 0.0, 1.0
    x = np.linspace(-4, 4, 1000)
    pdf = norm.pdf(x, loc=mu, scale=sigma)
    cdf = norm.cdf(x, loc=mu, scale=sigma)
    F_t = norm.cdf(thresh, loc=mu, scale=sigma)

    fig = plt.figure(figsize=(10, 8))

    # PDF with shaded area up to threshold
    ax1 = plt.subplot(2,1,1)
    ax1.plot(x, pdf, color= "#fc7202", label="PDF (standard normal)")
    mask = x <= thresh
    ax1.fill_between(x[mask], pdf[mask], alpha=0.3, color="#fc7202", label=f"Area up to threshold = {F_t:.4f}")
    ax1.axvline(thresh, linestyle="--", color="#570a6d", label=f"threshold = {thresh:.2f}")
    ax1.set_xlabel("x")
    ax1.set_ylabel("Density")
    ax1.legend(loc="best")

    # --- Bottom: CDF with marker at threshold ---
    ax2 = plt.subplot(2,1,2)
    ax2.plot(x, cdf, color= "#fc7202", label="CDF (standard normal)")
    ax2.axvline(thresh, linestyle="--", color="#570a6d")
    ax2.axhline(F_t, linestyle="--", color="#570a6d")
    ax2.plot([thresh], [F_t], marker="o", color="#570a6d")
    ax2.set_title(f"CDF; F(threshold) = {F_t:.4f}")
    ax2.set_xlabel("x")
    ax2.set_ylabel("F(x)")
    ax2.legend(loc="best")

    plt.tight_layout()
    plt.show()

interact(
    pdf_cdf_threshold_standard,
    thresh=FloatSlider(value=0.0, min=-4, max=4, step=0.1, description="threshold"),);

interactive(children=(FloatSlider(value=0.0, description='threshold', max=4.0, min=-4.0), Output()), _dom_clas…

Now that we understand what the CDF is, we can connect it to the Kolmogorov–Smirnov (K–S) test - a nonparametric test that compares two CDFs.

**What the K-S test does?**\
The idea of the test is simple:
- It compares the **empirical CDF** of your data (based on observed values)  
  with a **theoretical CDF** (for example, the CDF of a normal distribution).
- It then measures the **largest vertical distance (D)** between these two curves.

If this maximum difference $D$ is large, it suggests that the sample does **not** come from the specified distribution.

**Key assumption:** The theoretical CDF you compare against must be **fully specified**.  
That is, its parameters (like mean and standard deviation) should be known **before** looking at your data.

**The main problem:** In practice, when we test for *normality*, we almost never know the true mean and standard deviation of the population.  
We usually **estimate them from the same sample** that we are testing, for example:
```python
stat, p = kstest(data, 'norm', args=(np.mean(data), np.std(data)))
```
This is a common approach, but it changes the distribution of the test statistic under the null hypothesis.
In other words, the standard K-S critical values are no longer valid when we plug in estimated parameters.
This means that the regular K-S test becomes too strict (conservative), meaning, it might reject normality less often than it should even when the data are not perfectly normal.

In our case, this leads to two practical approaches for performing the K-S test:
1. Pass parameters (mean and standard deviation) via `args`  
2. Standardize the data first
> Data standardization transforms a variable into *z*-scores (standard scores).\
> When the population mean and the population standard deviation are unknown, the standard score may be estimated by using\
> the sample mean ($\bar{x}$) and sample standard deviation ($s$) as estimates of the population values.\
> In these cases, the *z*-score is given by:
>
>   \begin{equation} z = \frac{x - \bar{x}}{s} \end{equation}




#### <font color='#fc7202'>Task 3:</font>  
1. Standardize all numeric variables and store them in a new DataFrame called `wine_data_standard`.  
2. Visualize the variable `density` before and after standardization to see that the shape of the distribution stays the same, while the center and scale change.  
3. Run the Kolmogorov–Smirnov test for each numeric variable in two ways:  
   - On the **raw data**, using `args=(mean, sd)` to specify the parameters.  
   - On the **standardized data**, comparing directly against `'norm'` (the standard normal distribution).  

In [ ]:
# YOUR CODE HERE!

## Part 2: Simulating Continuous Data

Yesterday, we simulated data for a **discrete random variable** (the roll of a die).  
When we rolled the die only a few times (for example, 7 rolls), the results looked noisy and irregular,  
and the underlying distribution (where all outcomes are equally likely) was not yet visible.  
As we increased the number of rolls, however, the distribution began to *stabilize* and approach the expected uniform shape.

Today, we’ll do something very similar, but this time for a **continuous random variable**.  
Instead of discrete dice outcomes, we’ll simulate measurements that follow a **normal distribution**,  
with a **mean of 10.5** and a **standard deviation of 1.8**.


#### <font color='#fc7202'>Task 4:</font>  
1. Generate a **small sample** of size 15 from this distribution and plot its histogram.  
2. Then, generate a **larger sample**. Choose the size yourself (for example, a few hundred or a few thousand observations), and plot a second histogram on the same scale.  

> *Hint:* use `np.random.normal(mean, sd, size=n)` to generate the data.


In [ ]:
# YOUR CODE HERE!

#### <font color='#fc7202'>Task 5:</font>  
Now, let’s use the same normal model to estimate a probability through simulation.  
Assume that values **≤ 7.0** are considered unusually low for this measurement.  
What is the probability that the next observation will fall into this range?

1. Draw one sufficiently large synthetic sample from the same normal distribution.  
2. Count how many of your simulated values are ≤ 7.0.  
3. Compute the fraction of such observations (this represents your simulated probability).  
4. Compare your simulated result with the analytical probability obtained using  
\begin{equation} P(X \le 7.0) = \texttt{norm.cdf(7.0, loc=mean, scale=sd)} \end{equation}

> *Hint:* You can use `np.mean(sample <= 7.0)` to calculate the fraction of simulated values ≤ 7.0.



In [74]:
# YOUR CODE HERE!

#### <font color='#fc7202'>Task 6:</font>
Assume the true process follows a normal distribution with mean = 10.0 and standard deviation of 1.8.

You run an experiment with *n* = 600 observations and obtain a sample mean of 10.5.  
How surprising is this result if the true mean is really 10.0?

1. Simulate many experiments: in each repetition, draw `n = 600` values from `Normal(10.0, 1.8)`.  
2. Compute the sample mean for each repetition.  
3. Estimate the probability 
   \begin{equation} P(\bar{X} \ge 10.5 \mid \mu=10.0, \sigma=1.8, n=600) \end{equation}
   as the fraction of simulated means that are ≥ 10.5.  
4. Visualize the sampling distribution of the simulated means (histogram) and add a vertical line at 10.5.  


In [95]:
# YOUR CODE HERE!

In the previous task, we repeated the experiment many times and computed the sample mean each round.  
The histogram of those sample means looked bell-shaped.  

That is exactly the **Central Limit Theorem (CLT)** in action:
- Regardless of the original data’s distribution (as long as observations are independent and identically distributed (i.i.d.) with finite variance),
- the sampling distribution of the mean $\bar X$ becomes approximately **Normal** as $n$ grows,
- centered at the true mean $\mu$ with standard deviation $\sigma/\sqrt{n}$.

So, the bell shape you observed for the collection of sample means is *evidence of the CLT*.

#### <font color='#fc7202'>Task 7:</font>
Let’s test whether the sample mean looks approximately normal even when the underlying data are not normal.
We’ll use an exponential distribution.

In `Python`, we can simulate such data using:
```python
np.random.exponential(scale=1/lam, size=n)
```
where:
- `lam` is the rate parameter *λ*,
- `1/lam` is the mean (the `scale` in `NumPy`’s function), and
- `size` is the number of values you want to generate.

1. Choose a sample size *n* (for example, 600) and a rate *λ* (for example, 0.1).
2. Repeat the experiment many times (e.g., 20 000 repetitions): in each repetition, draw *n* values from `np.random.exponential(scale=1/lam, size=n)` and compute their sample mean.
3. Plot a histogram of all simulated sample means.

In [106]:
# YOUR CODE HERE!